In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

##### interact

In [3]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, 
        max_length=1000, 
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=100,
        top_p=0.9,
        temperature=0.6,
        #num_return_sequences=3,
        device='gpu',
        mask_token_id=tokenizer.mask_token_id,
        eos_token_id=tokenizer.eos_token_id,
        unk_token_id=tokenizer.unk_token_id,
        #pad_token_id=tokenizer.pad_token_id,
    )

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:How are you?
DialoGPT: I'm good. How are you?
>> User:Too. Where are you?
DialoGPT: You know, I don't know.
>> User:Do you know Bill?
DialoGPT: No. What's he like?
>> User:he is high and silent
DialoGPT: That's good.
>> User:I am not sure. Do you silent?
DialoGPT: Not really.


#### one line

In [4]:
user_text = "What's your best film?"
# encode the new user input, add the eos_token and return a tensor in Pytorch
new_user_input_ids = tokenizer.encode(user_text + tokenizer.eos_token, return_tensors='pt')
# append the new user input tokens to the chat history
bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
# generated a response while limiting the total chat history to 1000 tokens, 
chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
# pretty print last ouput tokens from bot
print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

DialoGPT: I don't know.


#### multiple lines

In [5]:
chat_history = "|0|1|What weather do you like?|1|1|Sunny.|0|1|Do you ready for walk?"
bot_input_ids = tokenizer.encode(chat_history + tokenizer.eos_token, return_tensors='pt')
# generated a response while limiting the total chat history to 1000 tokens, 
chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
# pretty print last ouput tokens from bot
print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

DialoGPT: I'm ready for walk.


#### updated chat history

In [6]:
updated_chat_history = chat_history + '|1|1|' + tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
updated_chat_history

"|0|1|What weather do you like?|1|1|Sunny.|0|1|Do you ready for walk?|1|1|I'm ready for walk."

#### interact with server

In [7]:
import requests

In [8]:
chat_history = "|0|1|What weather do you like?|1|1|Sunny.|0|1|Do you ready for swim?"
res = requests.post('http://192.168.1.23:8083/gpt', chat_history)
res.text

"I'm ready for swim."

#### extract phrases

In [13]:
outputs = [tokenizer.decode(x, skip_special_tokens=True) for x in chat_history_ids]
outputs = [x.split('|') for x in outputs]
outputs

[['',
  '0',
  '1',
  'What weather do you like?',
  '1',
  '1',
  'Sunny.',
  '0',
  '1',
  "Do you ready for walk?I'm ready for walk."]]